In [1]:
pip install cplex

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install docplex

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
from docplex.cp.model import *

In [4]:
model=CpoModel()

In [5]:
import pandas as pd

In [6]:
classes_to_take=[]
val="class"
val=input("Enter the course you would like to take ")
while val.lower()!="exit":
    classes_to_take.append(val)
    val=input("Enter the course you would like to take ")
    

Enter the course you would like to take CS115
Enter the course you would like to take CS100
Enter the course you would like to take EXIT


In [7]:
#constraint: for classes in classes_to_take, the value must be 1
df=pd.read_csv("/Users/alishakhan/Desktop/School/SPRING-22/MATH535/Project/Sample-Data.csv")
df.head()

,Course Name,CRN,Credits,Days,Time,Course Quality,Professor Difficulty
0,CS100,1,3,F,1350 - 1505,1,1
1,CS100,2,3,M,1515 - 1630,2,1
2,CS100,3,3,MW,1700 - 1815,3,3
3,CS115,4,3,TR,1825 - 1940,1,2
4,CS115,5,3,F,1350 - 1505,2,2


In [8]:
#creating binary vars for each class
classes=[]
for index, row in df.iterrows():
    classes.append(model.integer_var(min=0, max=1, name="crn"+str(row['CRN'])))    
    #print("crn"+str(row['CRN']))
df['Vars']=classes

In [9]:
df.head()

,Course Name,CRN,Credits,Days,Time,Course Quality,Professor Difficulty,Vars
0,CS100,1,3,F,1350 - 1505,1,1,"crn1 = intVar(0, 1)"
1,CS100,2,3,M,1515 - 1630,2,1,"crn2 = intVar(0, 1)"
2,CS100,3,3,MW,1700 - 1815,3,3,"crn3 = intVar(0, 1)"
3,CS115,4,3,TR,1825 - 1940,1,2,"crn4 = intVar(0, 1)"
4,CS115,5,3,F,1350 - 1505,2,2,"crn5 = intVar(0, 1)"


In [10]:
for x in classes_to_take:
    #add constraint that at most ONE of the classes with the course name can be taken
    model.add_constraint(df.loc[df['Course Name'] == x]['Vars'].sum()==1)
#don't have to do credit constraint because of the above constraint

In [11]:
#observe that constraints were added
model.print_information()

Model: <ipython-input-4-d82fdc81c514>
 - source file: <ipython-input-4-d82fdc81c514>
 - modeling time: 8.26 sec
 - number of integer variables:  5
 - number of interval variables: 0
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        2
 - number of root expressions:   2
 - number of expression nodes:   12
 - operations:                   equal: 2, plus: 3


In [12]:
#creating minimization function (course quality and prof difficulty)
min_func=0
for index, row in df.iterrows():
    min_func+=((5-row['Course Quality'])+row['Professor Difficulty'])*row['Vars']

min_func

In [13]:
#solving model
min_func=model.minimize(min_func)
model.add(min_func)
sol=model.solve()
sol.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 8 variables, 2 constraints
 ! Presolve      : 4 extractables eliminated
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 8.0 (before), 8.0 (after)
 !  . Memory usage      : 267.6 kB (before), 267.6 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          8                 -
 + New bound is 0
                        0          8    1            -
 + New bound is 5
                        0          8    1   F        -
 + New bound is 6
 *            10        6  0.03s        1      (gap is 40.00%)
 *             9       41  0.03s        1      (gap is 33.33%)
               9       61          1    1            -
 + New bound is 9 (gap is 0.00%)
 ! ----------------